In [2]:
import pandas as pd
from Bio import SeqIO
import sys

sys.path.insert(0, "../")

In [3]:
matrix_data = []
for record in SeqIO.parse("../../raw_sequenced_data/d_salina_scaffolds.denovo.augustus.abinitio.aa", "fasta"):
    row = [
       record.id,
       str(record.seq),
       len(str(record.seq)) 
    ]

    matrix_data.append(row)

df_p_salina = pd.DataFrame(data=matrix_data, columns=["id", "sequence", "len"])
df_p_salina

,id,sequence,len
0,g1.t1,MVNLTPWRKAQTCEDNGMGSSTATSTQEKDGPRPRWNLGILSDPQT...,743
1,g2.t1,MPLRDPMLPKKDISAVYDTPTVQLRTPEDNLTPWQYMTVSWMEPLI...,2458
2,g3.t1,MNGPSWMRRRPSNIGSGSIPARATLSTPMADGTHPYQDSPSIRLLS...,350
3,g4.t1,MASPQQIRTKLTDLLKIQHPVMLAGMNVAAGPKLAAAVTNAGGLGV...,358
4,g5.t1,MFTTASRVAALRARSRVAQPFAASSVSISGQAAQNFRALHSKRLAR...,547
...,...,...,...
9532,g9533.t1,MHRRALEGSENALGLEHPDTLTSVSQLGSVLSSQGKYDEAEAMHRR...,68
9533,g9534.t1,FLSLILAQQCRIATALEQVVGHLARMGGGLGGGDSDDDESEPDGLG...,69
9534,g9535.t1,MHRRDLEGSEKALGFEHPDTLTSVSQLGSVLSSQGKYDEAEAMHRR...,59
9535,g9536.t1,EDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDE,43


In [4]:
df_not_use = df_p_salina[(df_p_salina["len"]<50) | (df_p_salina["len"]>=1022)]
df_not_use

,id,sequence,len
1,g2.t1,MPLRDPMLPKKDISAVYDTPTVQLRTPEDNLTPWQYMTVSWMEPLI...,2458
20,g21.t1,MAANGQQVFDPVLAAHNTMASSADRAQKEQAHQFLEQFQKSQEAWT...,1518
24,g25.t1,MLEEMANVQAATSSGRPGDVSPIGNSGSVPNPKMRKRTKTGCLTCR...,1163
25,g26.t1,MDFPRGHAALPLTETDFLDFIYNYFKAEYYPIRSFWSTATAFPRQP...,1090
30,g31.t1,MGKHLDTRRIRSSTPPELRDYFKVLLENLSVESITQQLLQAVSIGS...,1671
...,...,...,...
9435,g9436.t1,MCANCDQPRRSVYSVFAT,18
9514,g9515.t1,AAINAAGWSVPPFLVFAGQYHLSAWYEEKDILRDWAIAVSDNG,43
9518,g9519.t1,MAFQGNAQNNVARGQGPAAPAQPRPENDEMLEASDSDTEDDPSARE,46
9535,g9536.t1,EDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDEDE,43


In [5]:
df_to_process = df_p_salina[(df_p_salina["len"]>=50) & (df_p_salina["len"]<1022)]
df_to_process

,id,sequence,len
0,g1.t1,MVNLTPWRKAQTCEDNGMGSSTATSTQEKDGPRPRWNLGILSDPQT...,743
2,g3.t1,MNGPSWMRRRPSNIGSGSIPARATLSTPMADGTHPYQDSPSIRLLS...,350
3,g4.t1,MASPQQIRTKLTDLLKIQHPVMLAGMNVAAGPKLAAAVTNAGGLGV...,358
4,g5.t1,MFTTASRVAALRARSRVAQPFAASSVSISGQAAQNFRALHSKRLAR...,547
5,g6.t1,MAAQELRYDGQTVVVTGAGGGLGREYAVFYGSRGANVVVNDLGGSF...,903
...,...,...,...
9530,g9531.t1,LGVGITVKALYNNTNSFDSIINLEKATTATNELNAFIRSNATELLV...,75
9531,g9532.t1,EHPDTLTSVSQLGSVLSSQGKYDEAEAMHRRALEGREKALGFEHPD...,78
9532,g9533.t1,MHRRALEGSENALGLEHPDTLTSVSQLGSVLSSQGKYDEAEAMHRR...,68
9533,g9534.t1,FLSLILAQQCRIATALEQVVGHLARMGGGLGGGDSDDDESEPDGLG...,69


In [6]:
df_to_process.to_csv("../../results/mapping_p_salina/filter_sequences.csv", index=False)

In [6]:
from coding_strategies.lib.apply_pretrained_models import using_bioembedding

bioembedding_instance = using_bioembedding(
    dataset=df_to_process,
    id_seq="id",
    column_seq="sequence",
    is_reduced=True,
    device="cuda"
)

In [7]:
df_prottrans_albert = bioembedding_instance.apply_prottrans_albert()
df_prottrans_albert.to_csv("../../results/mapping_p_salina/df_prottrans_albert.csv", index=False)

Some weights of the model checkpoint at /home/dmedina/.cache/bio_embeddings/prottrans_albert_bfd/model_directory were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.bias', 'predictions.dense.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Reducing embeddings: 846it [07:06,  1.98it/s]


KeyboardInterrupt: 

In [ ]:
df_prottrans_t5_uniref = bioembedding_instance.apply_prottrans_T5_UniRef()
df_prottrans_t5_uniref.to_csv("../../results/mapping_p_salina/df_prottrans_t5_uniref.csv", index=False)

In [ ]:
df_prottrans_t5_xlu50 = bioembedding_instance.apply_prottrans_T5_XLU50()
df_prottrans_t5_xlu50.to_csv("../../results/mapping_p_salina/df_prottrans_t5_xlu50.csv", index=False)